# Datos suministrados

In [2]:
import numpy as np
import sympy as sp

In [64]:
## Datos del codigo deestudiante 

x = 2 + 1 + 8 + 4 + 0 + 0 + 2 

## Cables.
# Longitudes de cables en [m]

CB = 1.5
DB = 1.5
EB = 1.5 

# Diametro de cables en pulgadas y convertido a metros.

diametro = (1/4) * (1/39.970)

# Diametro de pasadores en pulgadas y convertido a metros.

diametro_pasadores = (1/2) * (1/39.970)

# Longitud barra AB

L = (0.2) * x

# Area de cables 

area_cables = np.pi * (diametro/2) ** 2

# Area de pasadores 

area_pasadores = np.pi * (diametro_pasadores/2) ** 2

# Factor de Seguridad respecto al esfuerzo de fluencia 

FS = 1.17

# Factor de seguridad respecto al esfuerzo último

FS_ultimo = 1.20

In [65]:
## Propiedades del material

esfuerzo_fluencia_traccion = 250 #MPa
esfuerzo_fluencia_corte = 145 #MPa
esfuerzo_ultimo = 400 #MPa
E = 200 #(GPa)

In [66]:
# Angulos entre cables

angulo_BC = 40 * np.pi / 180
angulo_BE = 50 * np.pi / 180

## EQUILIBRIO. 

In [67]:
Ax = sp.symbols('Ax')
Ay = sp.symbols('Ay')
F_bc = sp.symbols('F_bc')
F_bd = sp.symbols('F_bd')
F_be = sp.symbols('F_be')
W = sp.symbols('W')

# ƩFx=0

Fx = Ax - F_bc * sp.sin(angulo_BC) + F_be * sp.sin(angulo_BE)
equilibrio_X = sp.Eq(Fx, 0)
equilibrio_X

Eq(Ax - 0.642787609686539*F_bc + 0.766044443118978*F_be, 0)

In [68]:
# ƩFy=0

Fy = Ay - W * L + F_bd * sp.cos(angulo_BC) + F_be * sp.cos(angulo_BE) + F_bd
equilibrio_Y = sp.Eq(Fy, 0)
equilibrio_Y

Eq(Ay + 1.76604444311898*F_bd + 0.642787609686539*F_be - 3.4*W, 0)

In [69]:
# ƩMA=0

M_a = - W * (L/2) + F_bc * sp.cos(angulo_BC) * L + F_be * sp.cos(angulo_BE) * L + F_bd * L
equilibrio_momentos = sp.Eq(M_a, 0)
equilibrio_momentos

Eq(2.60455110660453*F_bc + 3.4*F_bd + 2.18547787293423*F_be - 1.7*W, 0)

In [70]:
# Compatibilidad 

equivalencia_F_bc = sp.Eq(F_bc - F_bd * sp.cos(angulo_BC), 0)
expresion_equivalencia_F_bc = F_bd * sp.cos(angulo_BC)
equivalencia_F_bc

Eq(F_bc - 0.766044443118978*F_bd, 0)

In [71]:
equivalencia_F_be = sp.Eq(F_be - F_bd * sp.cos(angulo_BE), 0)
expresion_equivalencia_F_be = F_bd * sp.cos(angulo_BE)
equivalencia_F_be

Eq(-0.642787609686539*F_bd + F_be, 0)

In [72]:
# Ralacion entre la ecuacion de ƩMA=0 y Compatibilidad en terminos de F_bd

ecuacion = sp.Eq(M_a.subs([(F_bc, expresion_equivalencia_F_bc), (F_be, expresion_equivalencia_F_be)]), 0)
ecuacion

Eq(6.8*F_bd - 1.7*W, 0)

In [73]:
F_bd_vs_W = sp.solve(ecuacion)[0][F_bd]
F_bd_vs_W

0.25*W

In [74]:
# Dejar las incognitas F_bc y F_be en funcion de w

bc = expresion_equivalencia_F_bc.subs(F_bd, F_bd_vs_W)
be = expresion_equivalencia_F_be.subs(F_bd, F_bd_vs_W)
fuerzas_cables = [bc, F_bd_vs_W, be]
fuerzas_cables

[0.191511110779745*W, 0.25*W, 0.160696902421635*W]

In [86]:
Wmax = []
F_1 = sp.symbols('F_1')
for fuerza in fuerzas_cables:
    # Esfuerzo normal en los cables
    
    esfuerzo_normal = sp.Eq(F_1 / area_cables, esfuerzo_ultimo * 10 ** 6 / FS_ultimo)
    W_cab = sp.solve(esfuerzo_normal)[0] / 1000
    
    # Esfuero cortante en los pasadores C, D y E
    
    esfuerzo_cortante = sp.Eq(F_1 / area_pasadores, esfuerzo_fluencia_corte * 10 ** 6 / FS)
    W_pas = sp.solve(esfuerzo_cortante)[0] / 1000
    print(W_cab, W_pas)
    Wmax.append(W_cab)
    Wmax.append(W_pas)

print("Carga máxima que soportan los cables [kN / m]")
min(Wmax)
print()

10.2418956683405 15.2315371477884
10.2418956683405 15.2315371477884
10.2418956683405 15.2315371477884
Carga máxima que soportan los cables [kN / m]



In [79]:
# Analisis en los pasadores de la Barra AB

Ay_eq = Fy.subs([(F_bc, expresion_equivalencia_F_bc), (F_be, expresion_equivalencia_F_be), (F_bd, F_bd_vs_W)])
Ay_vs_W = sp.solve(sp.Eq(Ay_eq, 0))
Ay_vs_W = Ay_vs_W[0][Ay]
Ay_vs_W

2.85519491142862*W

In [80]:
Ax_eq = Fx.subs([(F_bc, expresion_equivalencia_F_bc), (F_be, expresion_equivalencia_F_be), (F_bd, F_bd_vs_W)])
Ax_vs_W = sp.solve(sp.Eq(Ax_eq, 0))
Ax_vs_W = Ax_vs_W[0][Ax]
Ax_vs_W

-2.77555756156289e-17*W

In [81]:
# FA = (Ax^2+Ay^2)^1/2

F_A = sp.sqrt(Ay_vs_W ** 2 + Ax_vs_W ** 2)
F_A

2.85519491142862*sqrt(W**2)

In [82]:
# Fuerza en B 

F_Bx = expresion_equivalencia_F_be.subs(F_bd, F_bd_vs_W) * sp.sin(angulo_BE) - expresion_equivalencia_F_bc.subs(F_bd, F_bd_vs_W) * sp.sin(angulo_BC)
F_By = expresion_equivalencia_F_be.subs(F_bd, F_bd_vs_W) * sp.cos(angulo_BE) + expresion_equivalencia_F_bc.subs(F_bd, F_bd_vs_W) * sp.cos(angulo_BC) + F_bd_vs_W
F_B = sp.sqrt(F_Bx ** 2 + F_By ** 2)
F_B

0.5*sqrt(W**2)

In [85]:
fuerzas_apoyos = [F_A, F_B]
w_max_apoyos = []
for fuerza in fuerzas_apoyos:
    esfuerzo_cortante = sp.Eq(fuerza / area_pasadores, FS_ultimo * 10 ** 6 / FS)
    w_pasadores = sp.solve(esfuerzo_cortante)[0] / 1000
    w_max_apoyos.append(w_pasadores)

print("Carga máxima que soportan los pasadores [kN / m]")
carga_maxima = min(Wmax)
carga_maxima

Carga distribuida máxima por pasadores [kN / m]


10.2418956683405

# Con la carga 10.2418956683405 [kN / m] (del numeral anterior), determine las reacciones, fuerzas ydeformaciones de todos los cables.

In [87]:
# Fuerza FBE y la deformacuon correspondiente 

valor_F_be = expresion_equivalencia_F_be.subs([(F_bd, F_bd_vs_W), (W, carga_maxima)])
deformacion_F_be = (valor_F_be * 1000 * EB) / (area_cables * E * 10 ** 9)
# fuerza en kN y deformacion en mm
valor_F_be, deformacion_F_be * 1000

(1.64584090882787, 0.401742256054086)

In [88]:
# Fuerza FBC y la deformacuon correspondiente 

valor_F_bc = expresion_equivalencia_F_bc.subs([(F_bd, F_bd_vs_W), (W, carga_maxima)])
deformacion_F_bc = (valor_F_bc * 1000 * CB) / (area_cables * E * 10 ** 9)
valor_F_bc, deformacion_F_bc * 1000

(1.96143681593413, 0.478777776949361)

In [89]:
# Fuerza FBD y la deformacuon correspondiente 

valor_F_bd = F_bd_vs_W.subs(W, carga_maxima)
deformacion_F_bd = (valor_F_bd * 1000 * DB) / (area_cables * E * 10 ** 9)
valor_F_bd, deformacion_F_bd * 1000

(2.56047391708511, 0.624999999999999)

In [90]:
# Reaccion Ax

valor_Ax = Fx.subs([(F_be, valor_F_be), (F_bc, valor_F_bc)])
valor_Ax = sp.solve(sp.Eq(valor_Ax, 0))[0]
valor_Ax

-4.44089209850063e-16

In [91]:
# Reaccion Ay

valor_Ay = Fy.subs([(F_be, valor_F_be), (F_bc, valor_F_bc), (F_bd, F_bd_vs_W), (W, carga_maxima)])
valor_Ay = sp.solve(sp.Eq(valor_Ay, 0))[0]
valor_Ay

29.2426083956285